# 1.) Import the modules we will need

In [1]:
from __future__ import division, unicode_literals
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings("ignore")

# 2.) Take a look at the files we have to work with

In [2]:
glob.glob('./input/sales-*.xlsx')

[u'./input/sales-feb-2014.xlsx',
 u'./input/sales-jan-2014.xlsx',
 u'./input/sales-mar-2014.xlsx']

## a.) Now let's grab all the files and concatenate them together

In [3]:
all_data = pd.DataFrame()
for f in glob.glob('./input/sales-*.xlsx'):
    df = pd.read_excel(f)
    all_data = pd.concat([all_data, df])
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1742 entries, 0 to 1499
Data columns (total 8 columns):
account number    1742 non-null int64
date              1742 non-null object
discount          1500 non-null float64
ext price         1742 non-null float64
name              1742 non-null object
quantity          1742 non-null int64
sku               1742 non-null object
unit price        1742 non-null float64
dtypes: float64(3), int64(2), object(3)
memory usage: 122.5+ KB


## b.) We can get quick stats on numeric columns just by running a "describe"

In [4]:
all_data.describe()

,account number,discount,ext price,quantity,unit price
count,1742.000000,1500.00,1742.000000,1742.000000,1742.000000
mean,485766.487945,0.05,1349.229392,24.319173,54.985454
std,223750.660792,0.00,1094.639319,14.502759,26.108490
min,141962.000000,0.05,-97.160000,-1.000000,10.030000
25%,257198.000000,0.05,468.592500,12.000000,32.132500
50%,527099.000000,0.05,1049.700000,25.000000,55.465000
75%,714466.000000,0.05,2074.972500,37.000000,77.607500
max,786968.000000,0.05,4824.540000,49.000000,99.850000


## c.) Let's look at the raw data

In [5]:
all_data.head()

,account number,date,discount,ext price,name,quantity,sku,unit price
0,383080,2014-02-01 09:04:59,NaN,235.83,Will LLC,7,B1-20000,33.69
1,412290,2014-02-01 11:51:46,NaN,232.32,Jerde-Hilpert,11,S1-27722,21.12
2,412290,2014-02-01 17:24:32,NaN,107.97,Jerde-Hilpert,3,B1-86481,35.99
3,412290,2014-02-01 19:56:48,NaN,1814.70,Jerde-Hilpert,23,B1-20000,78.90
4,672390,2014-02-02 03:45:20,NaN,2679.36,Kuhn-Gusikowski,48,S1-06532,55.82


In [6]:
all_data.tail()

,account number,date,discount,ext price,name,quantity,sku,unit price
1495,737550,2014-12-30 13:38:13,0.05,669.60,"Fritsch, Russel and Anderson",12,S1-06532,55.80
1496,786968,2014-12-30 21:42:17,0.05,486.18,"Frami, Hills and Schmidt",37,S1-06532,13.14
1497,239344,2014-12-30 22:45:19,0.05,542.50,Stokes LLC,14,S2-10342,38.75
1498,642753,2014-12-31 10:36:24,0.05,197.91,Pollich LLC,3,S2-82423,65.97
1499,383080,2014-12-31 12:48:35,0.05,1764.72,Will LLC,38,S2-00301,46.44


# 3.) Time to do a little house cleaning and transformation

## a.) First the clean up

In [7]:
all_data['discount'].fillna(0, inplace=True)          #  Let's fill mising values with zeros
all_data['ext price'] = abs(all_data['ext price'])    #  Let's make the negative numbers go bye bye
all_data['unit price'] = abs(all_data['unit price'])
all_data['quantity'] = abs(all_data['quantity'])
all_data['date'] = pd.to_datetime(all_data['date'])   #  Let's make this a date object so we can do fancy stuff
all_data.head()

,account number,date,discount,ext price,name,quantity,sku,unit price
0,383080,2014-02-01 09:04:59,0,235.83,Will LLC,7,B1-20000,33.69
1,412290,2014-02-01 11:51:46,0,232.32,Jerde-Hilpert,11,S1-27722,21.12
2,412290,2014-02-01 17:24:32,0,107.97,Jerde-Hilpert,3,B1-86481,35.99
3,412290,2014-02-01 19:56:48,0,1814.70,Jerde-Hilpert,23,B1-20000,78.90
4,672390,2014-02-02 03:45:20,0,2679.36,Kuhn-Gusikowski,48,S1-06532,55.82


## b.) Now let's add a couple calculated columns

In [8]:
all_data['total price'] = all_data['unit price'] * all_data['quantity'] * (1 - all_data['discount'])
all_data['month'] = all_data['date'].dt.month
all_data.tail()

,account number,date,discount,ext price,name,quantity,sku,unit price,total price,month
1495,737550,2014-12-30 13:38:13,0.05,669.60,"Fritsch, Russel and Anderson",12,S1-06532,55.80,636.1200,12
1496,786968,2014-12-30 21:42:17,0.05,486.18,"Frami, Hills and Schmidt",37,S1-06532,13.14,461.8710,12
1497,239344,2014-12-30 22:45:19,0.05,542.50,Stokes LLC,14,S2-10342,38.75,515.3750,12
1498,642753,2014-12-31 10:36:24,0.05,197.91,Pollich LLC,3,S2-82423,65.97,188.0145,12
1499,383080,2014-12-31 12:48:35,0.05,1764.72,Will LLC,38,S2-00301,46.44,1676.4840,12


# The road so far
* Merged 3 data files of different format and scale
* Cleaned and normalized data
* Added calculated columns
* Only 11 lines of "real" code used so far

# 4.) Let's join our data to another data set

In [9]:
customer_data = pd.read_excel('./input/customer-status.xlsx')
customer_data.head()

,account number,name,address,city,state,status
0,740150,Barton LLC,123 Main st,Norfolk,Virginia,gold
1,714466,Trantow-Barrows,467 Some Rd,Dover,Delawhere?,silver
2,218895,Kulas Inc,155 Baker St,Lenordtown,Maryland,bronze
3,307599,"Kassulke, Ondricka and Metz",101 Wall St,Richmond,Verginya,bronze
4,412290,Jerde-Hilpert,302 Broadway,Celebration,Florda,bronze


In [10]:
all_data = pd.merge(all_data, customer_data, how='left')    #  Just like in SQL, we can left join data sets
all_data.head()

,account number,date,discount,ext price,name,quantity,sku,unit price,total price,month,address,city,state,status
0,383080,2014-02-01 09:04:59,0,235.83,Will LLC,7,B1-20000,33.69,235.83,2,NaN,NaN,NaN,NaN
1,412290,2014-02-01 11:51:46,0,232.32,Jerde-Hilpert,11,S1-27722,21.12,232.32,2,302 Broadway,Celebration,Florda,bronze
2,412290,2014-02-01 17:24:32,0,107.97,Jerde-Hilpert,3,B1-86481,35.99,107.97,2,302 Broadway,Celebration,Florda,bronze
3,412290,2014-02-01 19:56:48,0,1814.70,Jerde-Hilpert,23,B1-20000,78.90,1814.70,2,302 Broadway,Celebration,Florda,bronze
4,672390,2014-02-02 03:45:20,0,2679.36,Kuhn-Gusikowski,48,S1-06532,55.82,2679.36,2,185 South Park,Huston,Texes,silver


## a.) Looks like we have some missing values again, let's fix that

In [11]:
all_data['status'].fillna('bronze',inplace=True)    #  Let's replace missing values with 'bronze' as a default
all_data['address'].fillna('?',inplace=True)        #  Let's replace missing values with '?' as a default
all_data['city'].fillna('?',inplace=True)           #  Let's replace missing values with '?' as a default
all_data['state'].fillna('?',inplace=True)          #  Let's replace missing values with '?' as a default
all_data.head()

,account number,date,discount,ext price,name,quantity,sku,unit price,total price,month,address,city,state,status
0,383080,2014-02-01 09:04:59,0,235.83,Will LLC,7,B1-20000,33.69,235.83,2,?,?,?,bronze
1,412290,2014-02-01 11:51:46,0,232.32,Jerde-Hilpert,11,S1-27722,21.12,232.32,2,302 Broadway,Celebration,Florda,bronze
2,412290,2014-02-01 17:24:32,0,107.97,Jerde-Hilpert,3,B1-86481,35.99,107.97,2,302 Broadway,Celebration,Florda,bronze
3,412290,2014-02-01 19:56:48,0,1814.70,Jerde-Hilpert,23,B1-20000,78.90,1814.70,2,302 Broadway,Celebration,Florda,bronze
4,672390,2014-02-02 03:45:20,0,2679.36,Kuhn-Gusikowski,48,S1-06532,55.82,2679.36,2,185 South Park,Huston,Texes,silver


# 5.) Lets try to join another dirty data set to get location info

## a.) Insert a blank column as a placeholder

In [12]:
all_data.insert(13, "abbrev", np.nan)

## b.) Open a JSON file that has abbreviations for all the states

In [13]:
import json
states = json.loads(open('./input/state_abbreviations.json').read())

## c.) Create a function we will use to lookup state abbreviations

In [14]:
from fuzzywuzzy import process
def convert_state(row):
    abbrev = process.extractOne(row["state"], choices=states.keys(), score_cutoff=80)
    if abbrev:
        return states[abbrev[0]]
    return np.nan

## d.) Let's apply our function to populate the correct abbreviations

In [15]:
all_data['abbrev'] = all_data.apply(convert_state, axis=1)
all_data.head()

,account number,date,discount,ext price,name,quantity,sku,unit price,total price,month,address,city,state,abbrev,status
0,383080,2014-02-01 09:04:59,0,235.83,Will LLC,7,B1-20000,33.69,235.83,2,?,?,?,NaN,bronze
1,412290,2014-02-01 11:51:46,0,232.32,Jerde-Hilpert,11,S1-27722,21.12,232.32,2,302 Broadway,Celebration,Florda,FL,bronze
2,412290,2014-02-01 17:24:32,0,107.97,Jerde-Hilpert,3,B1-86481,35.99,107.97,2,302 Broadway,Celebration,Florda,FL,bronze
3,412290,2014-02-01 19:56:48,0,1814.70,Jerde-Hilpert,23,B1-20000,78.90,1814.70,2,302 Broadway,Celebration,Florda,FL,bronze
4,672390,2014-02-02 03:45:20,0,2679.36,Kuhn-Gusikowski,48,S1-06532,55.82,2679.36,2,185 South Park,Huston,Texes,TX,silver


# The road so far
* Merged 3 data files of different format and scale
* Cleaned and normalized data
* Added calculated columns
* Left joined 4th data file, and filled in missing values
* Built function to auto-magically populate column based on dirty data
* Only 25 lines of "real" code used so far

# 6.) Now that our data is combined and cleaned up, let's pivot!

In [16]:
pt = pd.pivot_table(
    all_data,                            #  This is the Dataframe we will pivot
    index=["name", 'sku'],               #  These are the fields, in order, that we want to pivot
    columns=['month'],                   #  Let's group by this these columns
    values=['quantity', 'total price'],  #  These are the value we want to agg
    aggfunc=[np.sum],                    #  This is the methods of aggrigation we want to use
    fill_value=0                         #  Since we know this will happen, let's fill missing values with zero
)
pt

sum                           \
                                         quantity                            
month                                          1    2   3   4   5   6   7    
name                            sku                                          
Barton LLC                      B1-04202        0    0   0   0   0   0   0   
                                B1-05914        0    0   0   0  24   0   0   
                                B1-20000       78   56  20   0   0   0   0   
                                B1-33087        0   70   0   0   0   9   0   
                                B1-33364       12    0   0   0   0   0   0   
                                B1-38851        0   40   0  42   5   0   0   
                                B1-50809       16    0   0  75  31   0   0   
                                B1-53102        2    0   0   0   0   0   0   
                                B1-53636        0    0   0  31   0   0   0   
                                B1-65551        0    0   0   7  38   0   0   
                                B1-69924        0    0   0   1  38   0   0   
                                B1-86481       40    0   0   0   0   0   0   
                                S1-06532       20    0   0  49  27  45  68   
                                S1-27722        0    0   0   0   0  38   0   
                                S1-30248        0   42   0   0   0   0   0   
                                S1-47412       54    0   0   0   0   0  40   
                                S1-50961        0    0   0   0   0   0   0   
                                S1-65481        0    0   0   0   0   0   0   
                                S1-82801        0    0  29   0   0   0  34   
                                S1-93683        0   42   0   0   0   0   0   
                                S2-10342        0   94   0   0  37  40   0   
                                S2-11481        0    0   0   0   0  25   0   
                                S2-16558        4    0   0   0   0   0   0   
                                S2-34077        0    0   0   0   0   0   0   
                                S2-77896        0    0   0   0   0  52   0   
                                S2-78676        0    0   0   0   0   0   0   
                                S2-82423        0    0   0  33  10   0   0   
                                S2-83881        0    0  12   6   0   0   0   
Cronin, Oberbrunner and Spencer B1-04202        0    0   0   0   0   0   0   
                                B1-05914       16    0   0   0   0   0  30   
...                                           ...  ...  ..  ..  ..  ..  ..   
White-Trantow                   S2-82423        0    0  38  30   0   0  11   
                                S2-83881        0   84   0   0   0   0   0   
Will LLC                        B1-04202        0    0   0   0   6   0   0   
                                B1-05914      136    0   0   1   0   0   0   
                                B1-20000        0   22  45  45   0   0   0   
                                B1-33087        0    0  19   0   0   0   0   
                                B1-33364      138    0   0   0   0   0   0   
                                B1-38851        0    0   0   3   0   0   0   
                                B1-53102        0  154   0   0   0  32   0   
                                B1-53636       74    0   0   0   0   0   0   
                                B1-65551        0    0   0   0   0   0  41   
                                B1-69924        0   66   0   0  98   0   0   
                                B1-86481        0    0   0   0  22   0   0   
                                S1-06532        0    0   0   0  44   8   0   
                                S1-27722        0    0   0   0   0   0  51   
                                S1-30248        0    0   0  16   0   0   0   
                                S1-47412        0    0   0   0   0  

## a.) Did you know that you can cross section pivots? 

In [17]:
pt.xs('Will LLC', level=0)    #  Let's cut out just the data for one account

sum                                         ...                  \
         quantity                                         ...     total price   
month          1    2   3   4   5   6   7   8   9   10    ...              3    
sku                                                       ...                   
B1-04202        0    0   0   0   6   0   0   0   0   1    ...          0.0000   
B1-05914      136    0   0   1   0   0   0   0   0   0    ...          0.0000   
B1-20000        0   22  45  45   0   0   0   0   0   0    ...       1278.2250   
B1-33087        0    0  19   0   0   0   0   0  13  38    ...       1024.1570   
B1-33364      138    0   0   0   0   0   0   0   0   0    ...          0.0000   
B1-38851        0    0   0   3   0   0   0   0  22   0    ...          0.0000   
B1-53102        0  154   0   0   0  32   0   6   0   0    ...          0.0000   
B1-53636       74    0   0   0   0   0   0   0   0   0    ...          0.0000   
B1-65551        0    0   0   0   0   0  41   0   0   0    ...          0.0000   
B1-69924        0   66   0   0  98   0   0   0   0   0    ...          0.0000   
B1-86481        0    0   0   0  22   0   0   0   0   0    ...          0.0000   
S1-06532        0    0   0   0  44   8   0   0   0   0    ...          0.0000   
S1-27722        0    0   0   0   0   0  51   0   0   0    ...          0.0000   
S1-30248        0    0   0  16   0   0   0   0   0   0    ...          0.0000   
S1-47412        0    0   0   0   0   0   0  28   0  50    ...          0.0000   
S1-50961        0    0  37   7   0   0   0   0   0   0    ...       2966.2705   
S1-65481        0    0   0   0   0   0   6   0   0   0    ...          0.0000   
S1-82801        0   42  41   0   0   0   0   0   0   0    ...       1661.4550   
S1-93683       90   82  28   0   0  37   0   0   0   0    ...       2416.8760   
S2-00301        0    0   0   0   0  38   0   0   0   0    ...          0.0000   
S2-10342        0    0   0  15   0   0   0  10   0   0    ...          0.0000   
S2-11481       34    0   0   0   0   0   0   0   0   0    ...          0.0000   
S2-16558        0    0   0   0   0   0   6   0   0   0    ...          0.0000   
S2-23246       46   94   0   0   0   0   0   0   0   0    ...          0.0000   
S2-34077        0    0   0  19   0   0   0   0   0   0    ...          0.0000   
S2-77896       28    0   0   0   0   0   0   0  43  40    ...          0.0000   
S2-78676        0    0   0   0   0   0  38   0   0  38    ...          0.0000   
S2-83881       70    0   0   0   7   0   0   0   0   0    ...          0.0000   

                                                                          \
                                                                           
month            4          5          6          7        8          9    
sku                                                                        
B1-04202     0.0000   346.6170     0.0000     0.0000    0.000     0.0000   
B1-05914    25.3840     0.0000     0.0000     0.0000    0.000     0.0000   
B1-20000  2272.1150     0.0000     0.0000     0.0000    0.000     0.0000   
B1-33087     0.0000     0.0000     0.0000     0.0000    0.000   966.0170   
B1-33364     0.0000     0.0000     0.0000     0.0000    0.000     0.0000   
B1-38851   158.2035     0.0000     0.0000     0.0000    0.000  1243.1320   
B1-53102     0.0000     0.0000  1368.9120     0.0000  241.338     0.0000   
B1-53636     0.0000     0.0000     0.0000     0.0000    0.000     0.0000   
B1-65551     0.0000     0.0000     0.0000  2321.8095    0.000     0.0000   
B1-69924     0.0000  5859.4575     0.0000     0.0000    0.000     0.0000   
B1-86481     0.0000  1681.8230     0.0000     0.0000    0.000     0.0000   
S1-06532     0.0000  3009.5240   139.4600     0.0000    0.000     0.0000   
S1-27722     0.0000     0.0000     0.0000  4268.4545    0.000     0.0000   
S1-30248   926.7440     0.0000     0.0000     0.0000    0.000     0.0000   
S1-47412     0.0000     0.0000     0.0000     0.0000 

# 7.) Now let's write all this data to the files we need

## a.) An Excel file with the full data set

In [19]:
all_data.to_excel('./output/all_data.xls')

## b.) An Excel file of the pivoted data

In [20]:
pt.to_excel('./output/pivot_data.xls')

## c.) An Excel file with a different cross-section on each tab

In [22]:
writer = pd.ExcelWriter('./output/pivoted_x-sections.xlsx')
for name in pt.index.get_level_values(0).unique():
    temp_df = pt.xs(name, level=0)
    temp_df.to_excel(writer,name)
writer.save()

# All said and done
* Merged 3 data files of different format and scale
* Cleaned and normalized data
* Added calculated columns
* Left joined 4th data file, and filled in missing values
* Built function to auto-magically populate column based on dirty data
* Dumped all data to flat excel file
* Built pivot table and exported to excel
* Built x-sectioned file of pivots and exported to excel
* Only 35 lines of "real" code used (note including imports and exploration steps)